In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, rosen, rosen_der
from sklearn.datasets import make_blobs
from sklearn import svm

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)


def scalar_pr(a,b):
    t = a*b
    return sum(t[i] for i in range(len(a)))

def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else: 
        return 0

In [2]:
trans = pd.read_csv('transactions.csv')
X_0 = pd.read_csv('data_00.csv')
X_1 = pd.read_csv('data_01.csv')
X_2 = pd.read_csv('data_02.csv')


In [20]:
#исходные данные
trans.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


In [25]:
#X_0, x_1, X_2 - панельные данные, X_0 и X_1 - с метками, метки X_2 нужно построить
X_0

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
#y_0, y_1 - метки класса
y_0 = X_0['pre']
del X_0['pre']
y_1 = X_1['pre']
del X_1['pre']
y_2 = X_2['pre']
del X_2['pre']
del X_0['4829']
del X_1['4829']
del X_2['4829']
w_0 = np.array([i for i in X_0.columns])
w_1 = np.array([i for i in X_1.columns])
w_2 = np.array([i for i in X_2.columns])

In [6]:
def return_SVC(X_1, y_1):
        
    clf = svm.SVC(C = 1, kernel = 'linear')
    clf.fit(X_1, y_1) 
    x1_duall = np.array(clf.dual_coef_)
    a = np.zeros(len(y_1))
    for i in range(len(clf.support_)):
        a[clf.support_[i]] = x1_duall[0][i]
    return a

In [8]:
clf = svm.SVC(C = 0.3, kernel = 'linear')
clf.fit(X_1, y_1) 


SVC(C=0.3, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
y_pred = clf.predict(X_2)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_1[:100])

0.65000000000000002

In [12]:
from sklearn import svm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, rosen, rosen_der
from sklearn.datasets import make_blobs

class SVM_panel():
    
    def __init__(self, C = 10, reg = 1, maxiter = 10000, ftol = 1e-06, method = 'SLSQP'):
        
        self.C = C
        self.reg = reg
        self.dual_coeff = None
        self.options = {'maxiter':maxiter,
                           'ftol': ftol}
        self.method = method
        
    
    
    def return_SVC(self, X_1, y_1):
        
        
        self.clf = svm.SVC(C = 1, kernel = 'linear')
        clf.fit(X_1, y_1) 
        x1_duall = np.array(clf.dual_coef_)
        a = np.zeros(len(y_1))
        for i in range(len(clf.support_)):
            a[clf.support_[i]] = x1_duall[0][i]
        return a
        
    def get_func_to_min(self, X, y):
        
        a_1 = self.return_SVC(X_0, y_0)
        return lambda lam:sum(-lam[i] 
                              for i in range(len(y)))+0.5*sum(sum(lam[i]*lam[j]*y[i]*y[j]*scalar_pr(X[i],X[j]) for i in range(len(y))) for j in range(len(y))) + self.reg*np.math.sqrt(sum((lam[i]*y[i] - a_1[i])**2 for i in range(len(y))))                  
        
        
    def get_bounds(self, y):   
       
        bnds = []
        for i in range(len(y)):
            bnds.append([0, C])
        return bnds
     
    def get_constraints(self, y):
        
        return {'type': 'eq', 
                'fun': lambda lam: sum(lam[i]*y[i] for i in range(len(y)))}                
        
        
    def get_initial(self, y):    
        
        initial = []
        for i in range(len(y)):
            initial.append(100)
        
        return initial
    
    
        
    def fit(self, data_train, labels):
        
        self.dual_coeff = minimize(self.get_func_to_min(data_train, labels), 
                                   self.get_initial(labels), 
                                   method = self.method,
                                   options = self.options,
                                   bounds = self.get_bounds(labels),
                                   constraints = self.get_constraints(labels))
        #for i in range(len(y)):
         #   if self.dual_coeff.x[i] > 0:
        #        print (self.dual_coeff.x[i])
        return self
        
        
        
    def return_omega_0(self, t, data_train, labels):

        rule = []
        omega = sum(t[i]*labels[i]*data_train[i] for i in range(len(labels)))
        for i in range(len(labels)):
            if (t[i] > 0.000001):
                rule.append(scalar_pr(omega, data_train[i]) - labels[i])
        omega_0 = rule[len(rule)//2]

        return omega_0

    def predict_index(self, data_train, labels, data_test):
        
        w_0 = self.return_omega_0(self.dual_coeff.x, data_train, labels)
        final_predict = []
        for j in range(len(data_test)):
            
            final_predict.append(sign(sum(self.dual_coeff.x[i]*labels[i]*scalar_pr(data_train[i], data_test[j]) for i in range(len(labels))) - 
                    w_0))
            
        return final_predict        